<a href="https://colab.research.google.com/github/eunjoo-ny/ML-Code/blob/main/10_PyTorch_Lab_10_1_mnist_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random
import torch.nn.init


In [33]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
   torch.cuda.manual_seed_all(777)

In [34]:
#parameters
learning_rate= 0.001
training_epochs =15
batch_size = 100

In [35]:
#minst dataset
mnist_train =dsets.MNIST(root='MNIST_data/',
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [31]:
#dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)


In [21]:
#CNN Model (2 conv layers)
class CNN(torch.nn.Module):
      def __init__(self):
          super(CNN,self).__init__()
          self.layer1 = torch.nn.Sequential(
              torch.nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1),
              torch.nn.ReLU(),
              torch.nn.MaxPool2d(kernel_size=2, stride=2)
          )
          self.layer2 = torch.nn.Sequential(
              torch.nn.Conv2d(32,64,kernel_size=3, stride=1,padding=1),
              torch.nn.ReLU(),
              torch.nn.MaxPool2d(kernel_size=2, stride=2)
          )

          
          
          self.fc= torch.nn.Linear(4*4*128, 10, bias= True)
          torch.nn.init.xavier_uniform_(self.fc.weight)

          
          
          
      def forward(self,x):
          out = self.layer1(x)
          out = self.layer2(out)
          out = out.view(out.size(0),-1)
          out = self.fc(out)
          return out

In [22]:
model = CNN().to(device)

In [23]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [29]:
total_batch= len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost=0

    for X,Y in data_loader:
        X= X.to(device)
        Y= Y.to(device)

        hypothesis=model(X)
        cost=criterion( hypothesis,Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost +=cost/total_batch

    print('[Epoch:{:>4}] cost ={:>.9}'.format(epoch+1, avg_cost))
     #print('[Epoch: {:.4d}] cost = {:.6f}'.format(epoch + 1, avg_cost))

print("Learning Finished")

Learning started. It takes sometime.
[Epoch:   1] cost =0.227534682
[Epoch:   2] cost =0.0609463155
[Epoch:   3] cost =0.0465784222
[Epoch:   4] cost =0.0380074605
[Epoch:   5] cost =0.0312924199
[Epoch:   6] cost =0.0263720769
[Epoch:   7] cost =0.022658091
[Epoch:   8] cost =0.0184389651
[Epoch:   9] cost =0.0162469596
[Epoch:  10] cost =0.013554994
[Epoch:  11] cost =0.010464292
[Epoch:  12] cost =0.00921157375
[Epoch:  13] cost =0.00996156875
[Epoch:  14] cost =0.00744847534
[Epoch:  15] cost =0.00672241906
Learning Finished


In [30]:
with torch.no_grad():
     X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
     Y_test = mnist_test.test_labels.to(device)

     prediction= model(X_test)
     correct_prediction = torch.argmax(prediction, 1) ==Y_test
     accuracy = correct_prediction.float().mean()
     print('Accuracy:',accuracy.item())

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.988099992275238
